In [1]:
#!pip install dlib opencv-python imutils tqdm pandas numpy

In [2]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


CUDA Available: True
GPU Count: 1
GPU Name: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [3]:
import torch
print("Torch:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


Torch: 2.3.1+cu121
CUDA Available: True
Device: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [4]:
from insightface.app import FaceAnalysis

try:
    print("➡ Testing FaceAnalysis init…")
    app = FaceAnalysis(allowed_modules=['detection'])
    app.prepare(ctx_id=0)
    print("✔ Face detector OK")
except Exception as e:
    print("❌ Error:", e)


➡ Testing FaceAnalysis init…
❌ Error: This ORT build has ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. Since ORT 1.9, you are required to explicitly set the providers parameter when instantiating InferenceSession. For example, onnxruntime.InferenceSession(..., providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'], ...)


In [5]:
import cv2, pandas as pd, os

csv = r"E:\CVL\finalproject\Model Testing\Datasets\NeuralTextures.csv"
videos_root = r"E:\CVL\finalproject\Model Testing\Datasets"

df = pd.read_csv(csv)
row = df.iloc[0]
path = os.path.join(videos_root, str(row["File Path"]))

print("Video exists?", os.path.exists(path))
cap = cv2.VideoCapture(path)
print("Frames:", int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
cap.release()


Video exists? True
Frames: 303


In [6]:

#test_video = os.path.join(videos_root, "Deepfakes", "01_02__meeting_serious__YVGY8LOK.mp4")
#print(os.path.exists(test_video), test_video)
# -------- CONFIG --------
csv_path = r"E:\CVL\finalproject\Model Testing\Datasets\NeuralTextures.csv"
videos_root = r"E:\CVL\finalproject\Model Testing\Datasets"
out_root = "Datasets"

output_size = 256
absolute_num = 20
train_split = 0.7


In [7]:
#import os
#test_video = os.path.join(videos_root, "DeepFakeDetection", "01_02__meeting_serious__YVGY8LOK.mp4")
#print(os.path.exists(test_video), test_video)


In [8]:
from insightface.app import FaceAnalysis

app = FaceAnalysis(name='buffalo_l',
                   providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0)  # GPU enabled

print("InsightFace working correctly!")


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_use_max_workspace': '1', 'cudnn_conv1d_pad_to_nc1d': '0', 'tunable_op_enabled': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\Ariq Sudibyo/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0

In [9]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import insightface
from insightface.app import FaceAnalysis

# Initialize face detector
app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0)

print("FaceAnalysis initialized!")

def ensure_dirs():
    for sp in ["train", "valid"]:
        for lbl in ["real", "fake"]:
            os.makedirs(os.path.join(out_root, sp, lbl), exist_ok=True)

ensure_dirs()

def extract_faces(row):
    video_rel = row["File Path"]
    label = str(row["Label"]).lower()
    label_folder = "real" if label in ["real", "0"] else "fake"

    video_path = os.path.join(videos_root, video_rel)
    if not os.path.exists(video_path):
        return False, f"Missing video: {video_path}"

    split = "train" if hash(video_path) % 100 < train_split * 100 else "valid"
    out_dir = os.path.join(out_root, split, label_folder)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        return False, "No frames"

    # ----- NEW: fixed 10 evenly spaced frames -----
    num_frames = 10
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frame_indices = sorted(set(frame_indices))
    # ----------------------------------------------

    saved = 0
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue

        faces = app.get(frame)
        if len(faces) == 0:
            continue

        # largest face
        face = max(faces, key=lambda x: (x.bbox[2] - x.bbox[0]) * (x.bbox[3] - x.bbox[1]))
        x1, y1, x2, y2 = map(int, face.bbox)

        crop = frame[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        crop = cv2.resize(crop, (output_size, output_size))

        fname = f"{os.path.splitext(os.path.basename(video_path))[0]}_{idx:06d}.png"
        cv2.imwrite(os.path.join(out_dir, fname), crop)
        saved += 1

    cap.release()
    return True, f"{video_path}: saved {saved} faces"


def run():
    df = pd.read_csv(csv_path)
    res = []

    print(f"Starting extraction for {len(df)} videos...\n")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        ok, msg = extract_faces(row)
        res.append((ok, msg))

    print("\nExtraction Finished!")
    print("\nSample results:")
    for r in res[:5]:
        print(r)


run()


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_use_max_workspace': '1', 'cudnn_conv1d_pad_to_nc1d': '0', 'tunable_op_enabled': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\Ariq Sudibyo/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_use_max_workspace': '1', 'cudnn_conv1d_pad_to_nc1d': '0', 'tunable_op_enabled': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\Ariq Sudibyo/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
FaceAnalysis initialized!
Starting extraction for 1000 videos...



C:\Conda\envs\fakedetection\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [22:40<00:00,  1.36s/it]


Extraction Finished!

Sample results:
(True, 'E:\\CVL\\finalproject\\Model Testing\\Datasets\\NeuralTextures/000_003.mp4: saved 10 faces')
(True, 'E:\\CVL\\finalproject\\Model Testing\\Datasets\\NeuralTextures/001_870.mp4: saved 10 faces')
(True, 'E:\\CVL\\finalproject\\Model Testing\\Datasets\\NeuralTextures/002_006.mp4: saved 10 faces')
(True, 'E:\\CVL\\finalproject\\Model Testing\\Datasets\\NeuralTextures/003_000.mp4: saved 10 faces')
(True, 'E:\\CVL\\finalproject\\Model Testing\\Datasets\\NeuralTextures/004_982.mp4: saved 10 faces')
